In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from pycaret.classification import *

In [3]:
df=pd.read_csv(r"C:\Users\Zhiyong\Desktop\fraud\traindata_2017_v2.csv")
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()
df.columns = [col.replace(' ', '_') for col in df.columns]

In [4]:
df.columns

Index(['Flow_ID', 'Source_IP', 'Source_Port', 'Destination_IP',
       'Destination_Port', 'Protocol', 'Timestamp', 'Flow_Duration',
       'Total_Fwd_Packets', 'Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', 'Fwd_Packet_Length_Max',
       'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean',
       'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Flow_Bytes/s',
       'Flow_Packets/s', 'Flow_IAT_Mean', 'Flow_IAT_Min', 'Fwd_IAT_Mean',
       'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_Packets/s', 'Min_Packet_Length',
       'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Variance',
       'FIN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count',
       'URG_Flag_Count', 'Down/Up_Ratio', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Std',
       'Label', 'Prot

In [5]:
df

,Flow_ID,Source_IP,Source_Port,Destination_IP,Destination_Port,Protocol,Timestamp,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,...,Active_Min,Idle_Std,Label,Protocol_0,Protocol_6,Protocol_17,average_weighted_degree,average_page_rank,average_degree_centrality,average_k_core
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,2017-07-07 03:30:00,3,2,0,...,0.0,0.0,0,False,True,False,3148.5,0.023721,0.211921,5.5
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,2017-07-07 03:30:00,109,1,1,...,0.0,0.0,0,False,True,False,3160.5,0.023790,0.212894,6.5
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,2017-07-07 03:30:00,52,1,1,...,0.0,0.0,0,False,True,False,3160.5,0.023790,0.212894,6.5
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,2017-07-07 03:30:00,34,1,1,...,0.0,0.0,0,False,True,False,2340.5,0.020319,0.176665,5.0
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,2017-07-07 03:30:00,3,2,0,...,0.0,0.0,0,False,True,False,3156.0,0.023745,0.212505,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480378,192.168.10.14-205.174.165.73-53231-8080-6,192.168.10.14,53231,205.174.165.73,8080,6,2017-07-07 12:59:00,1030558,3,3,...,0.0,0.0,1,False,True,False,3415.0,0.023433,0.151149,7.0
480379,192.168.10.9-205.174.165.73-6490-8080-6,192.168.10.9,6490,205.174.165.73,8080,6,2017-07-07 12:59:00,1008819,3,3,...,0.0,0.0,1,False,True,False,3990.5,0.029832,0.198091,7.0
480380,192.168.10.5-205.174.165.73-52376-8080-6,192.168.10.5,52376,205.174.165.73,8080,6,2017-07-07 12:59:00,1032755,3,3,...,0.0,0.0,1,False,True,False,4287.5,0.029935,0.213284,7.0
480381,192.168.10.15-205.174.165.73-54013-8080-6,192.168.10.15,54013,205.174.165.73,8080,6,2017-07-07 12:59:00,1045087,3,3,...,0.0,0.0,1,False,True,False,5973.0,0.037905,0.259836,7.0


In [6]:
df['Label'].unique()

array([0, 1], dtype=int64)

In [8]:
# Setup environment on training data
clf_setup = setup(data=df, target='Label', session_id=123, verbose = False)

In [ ]:
# # Initialize setup with your dataset and 'Label' as target
# clf_setup = setup(data=df, 
#                   target='Label',
#                   session_id=123,
#                   ignore_features=['Flow_ID', 'Source_IP', 'Source_Port', 'Destination_IP',
#        'Destination_Port', 'Protocol', 'Timestamp'],
#                   feature_selection = True,
#                   normalize=True)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.218214 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13408
[LightGBM] [Info] Number of data points in the train set: 1602531, number of used features: 66
[LightGBM] [Info] Start training from score -0.541404
[LightGBM] [Info] Start training from score -7.065259
[LightGBM] [Info] Start training from score -2.883783
[LightGBM] [Info] Start training from score -1.380959
[LightGBM] [Info] Start training from score -5.404564
[LightGBM] [Info] Start training from score -2.297395
[LightGBM] [Info] Start training from score -6.031526
[LightGBM] [Info] Start training from score -5.978896
[LightGBM] [Info] Start training from score -12.207653
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

,Description,Value
0,Session id,123
1,Target,Label
2,Target type,Multiclass
3,Target mapping,"Benign: 0, Bot: 1, DDoS: 2, DDoS attacks-LOIC-HTTP: 3, DoS GoldenEye: 4, DoS Hulk: 5, DoS Slowhttptest: 6, DoS slowloris: 7, Heartbleed: 8"
4,Original data shape,"(2289330, 84)"
5,Transformed data shape,"(2289330, 16)"
6,Transformed train set shape,"(1602531, 16)"
7,Transformed test set shape,"(686799, 16)"
8,Ignore features,7
9,Numeric features,76


In [9]:
best_models = compare_models(n_select=5)  # Select top 5 performing models

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9999,1.0000,0.9999,0.9999,0.9999,0.9995,0.9995,8.8770
et,Extra Trees Classifier,0.9998,1.0000,1.0000,0.9998,0.9999,0.9995,0.9995,3.6140
catboost,CatBoost Classifier,0.9997,1.0000,0.9999,0.9997,0.9998,0.9991,0.9991,18.4180
rf,Random Forest Classifier,0.9995,1.0000,1.0000,0.9993,0.9997,0.9983,0.9983,4.5960
gbc,Gradient Boosting Classifier,0.9990,0.9996,0.9999,0.9989,0.9994,0.9970,0.9971,35.2390
lightgbm,Light Gradient Boosting Machine,0.9989,0.9999,0.9986,1.0000,0.9993,0.9966,0.9966,2.0690
dt,Decision Tree Classifier,0.9977,0.9981,0.9975,0.9997,0.9986,0.9931,0.9932,1.1860
knn,K Neighbors Classifier,0.9970,0.9985,0.9988,0.9975,0.9981,0.9908,0.9908,15.8870
ridge,Ridge Classifier,0.9965,1.0000,0.9961,0.9995,0.9978,0.9892,0.9892,1.0760
lda,Linear Discriminant Analysis,0.9965,1.0000,0.9961,0.9995,0.9978,0.9892,0.9893,2.1900


In [11]:
test_df=pd.read_csv(r"C:\Users\Zhiyong\Desktop\fraud\testdata_2018_v2.csv")

In [16]:
test_df = test_df.set_axis(['Flow_ID', 'Source_IP', 'Source_Port', 'Destination_IP',
       'Destination_Port', 'Protocol', 'Timestamp', 'Flow_Duration',
       'Total_Fwd_Packets', 'Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', 'Fwd_Packet_Length_Max',
       'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean',
       'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min', 'Flow_Bytes/s',
       'Flow_Packets/s', 'Flow_IAT_Mean', 'Flow_IAT_Min', 'Fwd_IAT_Mean',
       'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std',
       'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_Packets/s', 'Min_Packet_Length',
       'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Variance',
       'FIN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count',
       'URG_Flag_Count', 'Down/Up_Ratio', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Std',
       'Label', 'Protocol_0', 'Protocol_6', 'Protocol_17',
       'average_weighted_degree', 'average_page_rank',
       'average_degree_centrality', 'average_k_core'], axis=1)

In [18]:
ensemble_model = blend_models(estimator_list=best_models)
predictions = predict_model(ensemble_model, data=test_df)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9994,1.0000,1.0000,0.9993,0.9996,0.9983,0.9983
1,0.9999,1.0000,0.9998,1.0000,0.9999,0.9995,0.9995
2,0.9999,1.0000,0.9999,1.0000,0.9999,0.9997,0.9997
3,0.9993,1.0000,1.0000,0.9991,0.9995,0.9977,0.9977
4,0.9999,1.0000,0.9999,1.0000,0.9999,0.9997,0.9997
5,0.9999,1.0000,1.0000,0.9999,0.9999,0.9995,0.9995
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9999,1.0000,0.9999,1.0000,0.9999,0.9996,0.9996
8,0.9999,1.0000,1.0000,0.9999,0.9999,0.9996,0.9996


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9527,1.0000,1.0000,0.8637,0.9269,0.8921,0.8974


In [20]:
from sklearn.metrics import classification_report

print(classification_report(test_df['Label'], predictions['Label']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1343290
           1       1.00      1.00      1.00    575394

    accuracy                           1.00   1918684
   macro avg       1.00      1.00      1.00   1918684
weighted avg       1.00      1.00      1.00   1918684

